# Presentation

For those who know the <a href="https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life" target="_blank">game of life</a>, it's part of the <a href="https://en.wikipedia.org/wiki/Cellular_automaton" target="_blank">Cellular Automaton </a>. Today we gonna build a different concept of it. 

The idea is to check on a grid all pairs of nxn grids and change it if there is a pattern recognize. For example on the following matrix :

\begin{vmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 0
\end{vmatrix}

if we have a pattern saying that :
\begin{vmatrix}
1 & 0 \\
0 & 1 
\end{vmatrix} become 
\begin{vmatrix}
0 & 1 \\
1 & 0
\end{vmatrix}
we end up after 1 turn to :

\begin{vmatrix}
0 & 1 & 0 \\
1 & 0 & 0 \\
0 & 0 & 0
\end{vmatrix}

To avoid looping on pattern, we must move into the grid. We alternatively change the position where we look for pattern, 1 turn we follow red square, the next turn we check green squares

![grid_auto.png](grid_auto.png)

# Code

In [1]:
from PyQt5 import QtGui, QtWidgets, QtCore
import sys
import numpy as np

To determine patterns we cannot use numpy array as key in a dictionnary. So we gonna create a unique key based on the pattern. 
For this we can reshape the matrix to an array and do a product with a matrix of base 2 (as value can be only 0 or 1). For example the pattern:

\begin{vmatrix}
1 & 0 \\
0 & 1 
\end{vmatrix} becomes \begin{vmatrix}
1 \\ 0 \\ 0 \\ 1 
\end{vmatrix}
and if we do the dot with the weighted matrix we end up with :

\begin{equation*}
\begin{vmatrix}
1 \\ 0 \\ 0 \\ 1
\end{vmatrix} \times \begin{vmatrix}
8 \\ 4 \\ 2 \\ 1
\end{vmatrix} = 9
\end{equation*}

For every pattern, we gonna apply this calculation and link their key in a disctionnary. Like that we know that the pattern with key a becomes the pattern with key b. After that we have to create a function to go from key to the original matrix.

In [2]:
def decimal_to_array(nb):
    binary = bin(nb)[2:].zfill(4)
    binary = list(binary)
    arr = np.array(binary, dtype=np.int8).reshape(2,2)
    return arr

pattern = {}

m = np.array([8,4,2,1])

u = np.array([1,0,0,1]).dot(m)
v = np.array([0,1,1,0]).dot(m)
pattern[u] = v
pattern[v] = u

u2 = np.array([1,0,0,0]).dot(m)
v2 = np.array([0,0,0,1]).dot(m)
pattern[u2] = v2
pattern[v2] = u2

u3 = np.array([0,1,0,0]).dot(m)
v3 = np.array([0,0,1,0]).dot(m)
pattern[u3] = v3
pattern[v3] = u3

u4 = np.array([1,1,0,0]).dot(m)
v4 = np.array([0,0,1,1]).dot(m)
pattern[u4] = v4
pattern[v4] = u4

u5 = np.array([1,0,1,0]).dot(m)
v5 = np.array([0,1,0,1]).dot(m)
pattern[u5] = v5
pattern[v5] = u5

u6 = np.array([1,1,0,0]).dot(m)
v6 = np.array([0,0,1,1]).dot(m)
pattern[u6] = v6
pattern[v6] = u6

print(pattern)

{9: 6, 6: 9, 8: 1, 1: 8, 4: 2, 2: 4, 12: 3, 3: 12, 10: 5, 5: 10}


Now what we can do is to create a simple Interface filled with checkboxes and every 40ms, run the computation and display it with checkboxes checked or unchecked

In [3]:
class CheckBoxVideo(QtWidgets.QWidget):
    def __init__(self, W, H):
        super(CheckBoxVideo, self).__init__()
        self.width = min(W, 50)
        self.height = min(H, 50)
        self.init_size = 10
        self.size_box = 14
        self.status = True
        self.list_cb = []
        
        # creation of emptry grid
        self.grid = np.zeros((self.height, self.width))
        
        # we init a small parts in the middle of the grid with random values
        a = np.random.randint(2, size=(self.init_size, self.init_size))
        self.grid[ (self.height-self.init_size)//2:(self.height+self.init_size)//2 , (self.width-self.init_size)//2:(self.width+self.init_size)//2 ] = a
        
        self.interface()

    def interface(self):
        self.resize(self.width * self.size_box + 6, self.height * self.size_box + 6)
        self.setWindowTitle(u"Automate cellulaire")
        for j in range(self.height):
            for i in range(self.width):
                check_box = QtWidgets.QCheckBox(self)
                check_box.move(i * self.size_box, j * self.size_box)
                self.list_cb.append(check_box)

        self.timer = QtCore.QBasicTimer()
        self.timer.start(40, self) # 25 img/s

        self.show()

    def timerEvent(self, e):
        if self.status:
            for y in range(0, self.height, 2):
                for x in range(0, self.width, 2):
                    decimal_eq = self.grid[y:y + 2, x:x + 2].reshape(1, 4).dot(m)[0]
                    new_decimal = pattern.get(decimal_eq, None)
                    if new_decimal is not None:
                        self.grid[y:y + 2, x:x + 2] = decimal_to_array(new_decimal)
        else:
            for y in range(1, self.height - 1, 2):
                for x in range(1, self.width - 1, 2):
                    decimal_eq = self.grid[y:y + 2, x:x + 2].reshape(1, 4).dot(m)[0]
                    new_decimal = pattern.get(decimal_eq, None)
                    if new_decimal is not None:
                        self.grid[y:y + 2, x:x + 2] = decimal_to_array(new_decimal)
        self.status = not self.status

        for index, checkBox in enumerate(self.list_cb):
            x = index % self.width
            y = index // self.width
            if self.grid[y, x] == 1:
                checkBox.setCheckState(2)
            else:
                checkBox.setCheckState(0)

In [ ]:
appli = QtWidgets.QApplication(sys.argv)
ckbx = CheckBoxVideo(40,30)
sys.exit(appli.exec_())

![Cellular_Automata.png](Cellular_Automata.png)

And as expected, 25 times per seconds, the system evolved. It probably also exist specific pattern like gliders for the game of life. I let you tries this by yourself :) 